In [35]:
import json
import requests
import pandas as pd
import awswrangler as wr
import utils as U


class Dashboard:
    def __init__(self, report_id, id):
        self.report_id = report_id
        self.id = id
        self.project_info = U.get_project_info(self.report_id, self.id).to_dict(orient='records')[0]
        self.data = None
        self.data_dict = {}

    def update_cft(self):
        cft = self.data['data']['cft']['avg_cft']
        self.data_dict['avg_cost_for_two'] = cft

    def update_affluence(self):
        cft = self.data['data']['affluence']['affluence_index']
        self.data_dict['affluence'] = cft

    def update_location_score_weights_and_grouped_indexes(self):
        weights = json.dumps({
 'affluence_index': 0.19,
 'apartments_index': 0.19,
  'fashion_index': 0.19,
 'vibrancy': 0.14,
 'healthcare_index': 0.10,
 'company_index': 0.10,
 'malls_index': 0.05,
 'supermarket_index': 0.04,
 })
        self.data_dict['location_score_weights'] = weights
        indexes = json.dumps({'company_index': 3.89,
                              'connectivity_index': 4.62,
                              'education_index': 4.58,
                              'electronics_index': 4.78,
                              'entertainment_index': 5.0,
                              'fashion_index': 4.98,
                              'grocery_index': 4.98,
                              'healthcare_index': 4.82,
                              'home_decor_index': 4.56,
                              'leisure_index': 2.75,
                              'malls_index': 0.0,
                              'parks_index': 0.86,
                              'religious_index': 4.5,
                              'supermarket_index': 3.58,
                              'vibrancy': 4.01})
        self.data_dict['grouped_indexes'] = indexes

    def generate_report_data(self):
        eps = ["cft", "population", "companies", "demand_generator", "projects", "affluence", "income", "competition",
               "category_count", "property_price", "high_street",'shopping_mall' ]
        all_data = {"data": {}}
        for ep in eps:
            url = f"http://127.0.0.1:8000/site_report/{self.report_id}/{self.id}/{ep}"
            req = requests.get(url)
            try:
                resp = req.json()
            except Exception as e:
                print(url, e)
                raise e
            data = resp['data']
            all_data['data'][ep] = data
        self.data = all_data

    def populate_single_cells(self):
        with open("dashboard_copy.json") as f:
            data = json.load(f)
        for i in data:
            try:
                i['values'] = [[self.get_dictionary_value(self.data, i['values'][0][0])]]
            except Exception as e:
                print(e, i['values'][0][0])
        # self.report_worksheet.batch_update(data)

    def insert_pois_ranked(self):
        a = pd.DataFrame(self.data['data']['category_count']['data'])['top_pois']
        from functools import reduce
        df = pd.DataFrame(reduce(lambda a, b: a + b, a.values.tolist())).sort_values(by='reviews_per_day',
                                                                                     ascending=False)
        df_ = df.to_dict(orient='records')[:]
        pois = df_
        df = df[df['brand_id'] != 'N_A'].to_dict(orient='records')[:]
        df = json.dumps(df)
        self.data_dict['top_brands'] = df
        return pois

    def update_category_count(self):
        buff = self.data['data']['category_count']
        self.data_dict['pois'] = json.dumps(buff)

    def update_high_streets(self):
        high_streets = self.data['data']['high_street']
        high_streets = json.dumps(high_streets)
        self.data_dict['high_streets'] = high_streets

    def update_competition(self):
        comp = self.data['data']['competition']
        comp = json.dumps(comp)
        self.data_dict['competition'] = comp

    def update_malls(self):
        comp = self.data['data']['shopping_mall']
        comp = json.dumps(comp)
        self.data_dict['shopping_malls'] = comp
    

    def update_household_distribution(self):
        comp = self.data['data']['income']
        comp = json.dumps(comp)
        self.data_dict['household_distribution'] = comp

    def update_property_price(self):
        # comp = self.data['data']['property_price']['price']
        # comp = json.dumps(comp)
        # self.data_dict['property_prices'] = comp
        comp = self.data['data']['property_price']
        comp[0]['top_pois'] = comp[0]['top_pois'][:]
        comp = json.dumps(comp)
        self.data_dict['apartments'] = comp

    def update_projects(self):
        p = self.data['data']['projects']
        p['projects'] = p['projects'][:]
        self.data_dict['projects'] = json.dumps(p)

    def update_companies(self):
        companies = self.data['data']['companies']
        demand_generators = self.data['data']['demand_generator']
        demand_generators['companies'] = companies
        self.data_dict['demand_generators'] = json.dumps(demand_generators)

    def get_revenue_score(self):
        self.data_dict['revenue_score'] = 81.95

    def population_chart(self):
        pop = self.data['data']['population']
        pop = json.dumps(pop)
        self.data_dict['population'] = pop

    @staticmethod
    def get_dictionary_value(d, keys):
        keys = keys.split(".")
        for i in keys:
            d = d.get(i)
        return d

    def generate_report(self):
        self.generate_report_data()
        # self.populate_single_cells()
        self.update_cft()
        # self.update_location_score_weights_and_grouped_indexes()
        self.update_affluence()
        self.insert_pois_ranked()
        self.update_household_distribution()
        self.update_property_price()
        self.update_projects()
        self.update_malls()
        self.population_chart()
        self.update_competition()
        self.update_category_count()
        self.update_high_streets()
        # self.get_revenue_score()
        self.update_companies()


In [36]:
import utils as U
import importlib

importlib.reload(U)
all_reports = U.get_project_info()

In [37]:
all_reports=all_reports.query('catchment_type!="i1000mtb" and id not in ("129065_775925","130636_775869")')#.tail(4)
all_reports

,report_id,id,site_name,lat,lng,geometry,location,catchment_type,top_brands,pois,...,avg_cost_for_two,revenue_score,high_streets,shopping_malls,poi_counts,projects_counts,created_at,location_score_weights,grouped_indexes,demand_generators
0,1f78f3ac-d799-470b-9738-2c4da5053cee,130110_775547,"Orion Mall - Dr. Rajkumar Road, Rajaji Nagar I...",13.011027,77.554736,"POLYGON ((77.494736 13.050067, 77.493202 13.05...","Dr. Rajkumar Road, Rajaji Nagar Industrial Sub...",i15mind,"[{""id"": ""ChIJe9qj3xs9rjsRDN6CamLC-Co"", ""name"":...","{""count"": 1083, ""avg_number_of_reviews_per_day...",...,460.911885,87.916832,"[{""cluster_name"": ""1st Main Club Road Near Foo...","{""count"": 8, ""pois"": [{""id"": ""ChIJoRKfACMWrjsR...",None,None,1706512914,"{""affluence_index"": 0.19, ""apartments_index"": ...","{""company_index"": 4.26, ""connectivity_index"": ...","{""clinic"": 2093, ""college"": 413, ""hospital"": 6..."
1,2f78f3ac-d799-470b-9738-2c4da5053cee,130110_775547,"Orion Mall - Dr. Rajkumar Road, Rajaji Nagar I...",13.011027,77.554736,POLYGON ((77.55064773796063 13.010557348159388...,"Dr. Rajkumar Road, Rajaji Nagar Industrial Sub...",i500mtb,"[{""id"": ""ChIJe9qj3xs9rjsRDN6CamLC-Co"", ""name"":...","{""count"": 131, ""avg_number_of_reviews_per_day""...",...,838.805970,87.916832,"[{""cluster_name"": ""Dr. Rajkumar Road, Rajaji N...","{""count"": 1, ""pois"": [{""id"": ""ChIJCUMusVM9rjsR...",None,None,1706512950,"{""affluence_index"": 0.19, ""apartments_index"": ...","{""company_index"": 4.26, ""connectivity_index"": ...","{""clinic"": 25, ""college"": 7, ""hospital"": 57, ""..."
2,374c8386-5837-43b8-8409-c456849263e7,130009_776325,Orion Avenue - Second Cross N. Thyagaraju Layo...,13.000957,77.632571,POLYGON ((77.62847982126846 13.000488913937247...,"Second Cross N. Thyagaraju Layout, Sathya Naga...",i500mtb,"[{""id"": ""ChIJAfIetiRpUjoRKDXrDBoNP-0"", ""name"":...","{""count"": 24, ""avg_number_of_reviews_per_day"":...",...,500.000000,82.472917,[],"{""count"": 0, ""pois"": []}",None,None,1706684037,"{""affluence_index"": 0.19, ""apartments_index"": ...","{""company_index"": 4.01, ""connectivity_index"": ...","{""clinic"": 27, ""hospital"": 5, ""pharmacy"": 8, ""..."
3,2ef72172-e65c-47df-9f06-2a98c9f1fa94,130009_776325,Orion Avenue - Second Cross N. Thyagaraju Layo...,13.000957,77.632571,"POLYGON ((77.644571 13.032047, 77.643571 13.03...","Second Cross N. Thyagaraju Layout, Sathya Naga...",i15mind,"[{""id"": ""18767303"", ""name"": ""Imperio Restauran...","{""count"": 674, ""avg_number_of_reviews_per_day""...",...,498.559078,82.472917,"[{""cluster_name"": ""8th Main 3rd Phase Peenya I...","{""count"": 0, ""pois"": []}",None,None,1706684064,"{""affluence_index"": 0.19, ""apartments_index"": ...","{""company_index"": 4.01, ""connectivity_index"": ...","{""clinic"": 1313, ""college"": 135, ""hospital"": 3..."
4,5fca8372-db67-4e32-9c34-17818bca7c19,130552_777638,"Orion Uptown - Old Madras Road, Kattanallur,Sa...",13.055260,77.763894,"POLYGON ((77.773894 13.116797, 77.772949 13.11...","Old Madras Road, Kattanallur,Sannatammanahalli",i15mind,"[{""id"": ""ChIJdUCMjRcPrjsRjZ7tgReBR3M"", ""name"":...","{""count"": 85, ""avg_number_of_reviews_per_day"":...",...,535.593220,40.173626,[],"{""count"": 2, ""pois"": [{""id"": ""ChIJfU4i1c0PrjsR...",None,None,1706691413,"{""affluence_index"": 0.19, ""apartments_index"": ...","{""company_index"": 2.15, ""connectivity_index"": ...","{""clinic"": 187, ""college"": 67, ""hospital"": 89,..."
5,ed374c4c-6dc3-473b-867e-ff707e347939,130552_777638,"Orion Uptown - Old Madras Road, Kattanallur,Sa...",13.055260,77.763894,"POLYGON ((77.75979674534983 13.05479248128713,...","Old Madras Road, Kattanallur,Sannatammanahalli",i500mtb,"[{""id"": ""ChIJu_szAwoPrjsROOxOyHed4WU"", ""name"":...","{""count"": 15, ""avg_number_of_reviews_per_day"":...",...,862.500000,40.173626,[],"{""count"": 0, ""pois"": []}",None,None,1706691490,"{""affluence_index"": 0.19, ""apartments_index"": ...","{""company_index"": 2.15, ""connectivity

In [38]:
query = '''UPDATE ind_site_reports_v2
SET
    top_brands = :top_brands,
    pois = :pois,
    projects = :projects,
    apartments = :apartments,
    household_distribution = :household_distribution,
    competition = :competition,
    population = :population,
    affluence = :affluence,
    avg_cost_for_two = :avg_cost_for_two,
    high_streets = :high_streets,
    shopping_malls = :shopping_malls,
    demand_generators = :demand_generators
WHERE report_id = :report_id and id=:id;
'''

In [39]:
if __name__ == '__main__':
    con, cur = U.connect_to_db()

    for proj in all_reports.to_dict(orient='records')[:]:
        rep = Dashboard(report_id=proj['report_id'], id=proj['id'])
        rep.generate_report()
        query_params = rep.data_dict
        query_params['report_id'] = proj['report_id']
        query_params['id'] = proj['id']
        cur.execute(query, query_params)
        con.commit()
    cur.close()
    con.close()


In [40]:
df = U.get_all_reports()

In [41]:
df=df.query('catchment_type!="i1000mtb" and id not in ("129065_775925","130636_775869")')

In [42]:
df = df.drop(columns=['poi_counts', 'projects_counts', 'median_price'])
df

,report_id,id,site_name,lat,lng,geometry,location,catchment_type,top_brands,pois,...,population,affluence,avg_cost_for_two,revenue_score,high_streets,shopping_malls,created_at,location_score_weights,grouped_indexes,demand_generators
0,1f78f3ac-d799-470b-9738-2c4da5053cee,130110_775547,"Orion Mall - Dr. Rajkumar Road, Rajaji Nagar I...",13.011027,77.554736,"POLYGON ((77.494736 13.050067, 77.493202 13.05...","Dr. Rajkumar Road, Rajaji Nagar Industrial Sub...",i15mind,"[{""id"": ""ChIJe9qj3xs9rjsRDN6CamLC-Co"", ""name"":...","{""count"": 1229, ""avg_number_of_reviews_per_day...",...,"{""age_0_19"": 278764.7038758153, ""age_20_34"": 2...",3.675222,460.911885,87.916832,"[{""cluster_name"": ""1st Main Club Road Near Foo...","{""count"": 8, ""pois"": [{""id"": ""ChIJoRKfACMWrjsR...",1706512914,"{""affluence_index"": 0.19, ""apartments_index"": ...","{""company_index"": 4.26, ""connectivity_index"": ...","{""clinic"": 2093, ""college"": 413, ""hospital"": 6..."
1,2f78f3ac-d799-470b-9738-2c4da5053cee,130110_775547,"Orion Mall - Dr. Rajkumar Road, Rajaji Nagar I...",13.011027,77.554736,POLYGON ((77.55064773796063 13.010557348159388...,"Dr. Rajkumar Road, Rajaji Nagar Industrial Sub...",i500mtb,"[{""id"": ""ChIJe9qj3xs9rjsRDN6CamLC-Co"", ""name"":...","{""count"": 136, ""avg_number_of_reviews_per_day""...",...,"{""age_0_19"": 4275.960897406801, ""age_20_34"": 4...",4.127932,838.805970,87.916832,"[{""cluster_name"": ""Dr. Rajkumar Road, Rajaji N...","{""count"": 1, ""pois"": [{""id"": ""ChIJCUMusVM9rjsR...",1706512950,"{""affluence_index"": 0.19, ""apartments_index"": ...","{""company_index"": 4.26, ""connectivity_index"": ...","{""clinic"": 25, ""college"": 7, ""hospital"": 57, ""..."
2,374c8386-5837-43b8-8409-c456849263e7,130009_776325,Orion Avenue - Second Cross N. Thyagaraju Layo...,13.000957,77.632571,POLYGON ((77.62847982126846 13.000488913937247...,"Second Cross N. Thyagaraju Layout, Sathya Naga...",i500mtb,"[{""id"": ""ChIJAfIetiRpUjoRKDXrDBoNP-0"", ""name"":...","{""count"": 29, ""avg_number_of_reviews_per_day"":...",...,"{""age_0_19"": 4633.231310760981, ""age_20_34"": 4...",3.616629,500.000000,82.472917,[],"{""count"": 0, ""pois"": []}",1706684037,"{""affluence_index"": 0.19, ""apartments_index"": ...","{""company_index"": 4.01, ""connectivity_index"": ...","{""clinic"": 27, ""hospital"": 5, ""pharmacy"": 8, ""..."
3,2ef72172-e65c-47df-9f06-2a98c9f1fa94,130009_776325,Orion Avenue - Second Cross N. Thyagaraju Layo...,13.000957,77.632571,"POLYGON ((77.644571 13.032047, 77.643571 13.03...","Second Cross N. Thyagaraju Layout, Sathya Naga...",i15mind,"[{""id"": ""18767303"", ""name"": ""Imperio Restauran...","{""count"": 761, ""avg_number_of_reviews_per_day""...",...,"{""age_0_19"": 164433.15741358572, ""age_20_34"": ...",3.506014,498.559078,82.472917,"[{""cluster_name"": ""8th Main 3rd Phase Peenya I...","{""count"": 0, ""pois"": []}",1706684064,"{""affluence_index"": 0.19, ""apartments_index"": ...","{""company_index"": 4.01, ""connectivity_index"": ...","{""clinic"": 1313, ""college"": 135, ""hospital"": 3..."
4,5fca8372-db67-4e32-9c34-17818bca7c19,130552_777638,"Orion Uptown - Old Madras Road, Kattanallur,Sa...",13.055260,77.763894,"POLYGON ((77.773894 13.116797, 77.772949 13.11...","Old Madras Road, Kattanallur,Sannatammanahalli",i15mind,"[{""id"": ""ChIJdUCMjRcPrjsRjZ7tgReBR3M"", ""name"":...","{""count"": 98, ""avg_number_of_reviews_per_day"":...",...,"{""age_0_19"": 37307.81912734546, ""age_20_34"": 3...",3.134148,535.593220,40.173626,[],"{""count"": 2, ""pois"": [{""id"": ""ChIJfU4i1c0PrjsR...",1706691413,"{""affluence_index"": 0.19, ""apartments_index"": ...","{""company_index"": 2.15, ""connectivity_index"": ...","{""clinic"": 187, ""college"": 67, ""hospital"": 89,..."
5,ed374c4c-6dc3-473b-867e-ff707e347939,130552_777638,"Orion Uptown - Old Madras Road, Kattanallur,Sa...",13.055260,77.763894,"POLYGON ((77.75979674534983 13.05479248128713,...","Old Madras Road, Kattanallur,Sannatammanahalli",i500mtb,"[{""id"": ""ChIJu_szA

In [50]:
wr.catalog.delete_table_if_exists(database='datasets_prep', table='ind_site_reports_data_v2')

True

In [43]:
wr.s3.to_parquet(df, path="s3://tuzomldev/site_reports/", table="ind_site_reports_data_v2", database='datasets_prep',
                 dataset=True, mode='overwrite',
                 dtype={'report_id': 'string', 'id': 'string', 'site_name': 'string', 'lat': 'double',
                        'lng': 'double', 'catchment_type': 'string', 'top_brands': 'string', 'pois': 'string',
                        'projects': 'string', 'apartments': 'string', 'median_price': 'string',
                        'household_distribution': 'string', 'competition': 'string', 'population': 'string',
                        'affluence': 'double', 'avg_cost_for_two': 'double', 'revenue_score': 'double',
                        'high_streets': 'string', 'shopping_malls': 'string', 'poi_counts': 'string',
                        'projects_counts': 'string', 'created_at': 'bigint', "companies": "string"})

{'paths': ['s3://tuzomldev/site_reports/f6e20b6bcd7541179cb749801fcba323.snappy.parquet'],
 'partitions_values': {}}

In [12]:
df

,report_id,id,site_name,lat,lng,geometry,location,catchment_type,top_brands,pois,...,population,affluence,avg_cost_for_two,revenue_score,high_streets,shopping_malls,created_at,location_score_weights,grouped_indexes,demand_generators
0,1f78f3ac-d799-470b-9738-2c4da5053cee,130110_775547,"Orion Mall - Dr. Rajkumar Road, Rajaji Nagar I...",13.011027,77.554736,"POLYGON ((77.494736 13.050067, 77.493202 13.05...","Dr. Rajkumar Road, Rajaji Nagar Industrial Sub...",i15mind,"[{""id"": ""ChIJe9qj3xs9rjsRDN6CamLC-Co"", ""name"":...","{""count"": 1083, ""avg_number_of_reviews_per_day...",...,"{""age_0_19"": 278764.7038758153, ""age_20_34"": 2...",3.675222,460.911885,81.950000,"[{""cluster_name"": ""1st Main Club Road Near Foo...","{""count"": 8, ""pois"": [{""id"": ""ChIJoRKfACMWrjsR...",1706512914,"{""companies"": 0.11, ""vibrancy"": 0.16, ""transpo...","{""company_index"": 3.89, ""connectivity_index"": ...","{""clinic"": 2093, ""college"": 413, ""hospital"": 6..."
1,2f78f3ac-d799-470b-9738-2c4da5053cee,130110_775547,"Orion Mall - Dr. Rajkumar Road, Rajaji Nagar I...",13.011027,77.554736,POLYGON ((77.55064773796063 13.010557348159388...,"Dr. Rajkumar Road, Rajaji Nagar Industrial Sub...",i500mtb,"[{""id"": ""ChIJe9qj3xs9rjsRDN6CamLC-Co"", ""name"":...","{""count"": 131, ""avg_number_of_reviews_per_day""...",...,"{""age_0_19"": 4275.960897406801, ""age_20_34"": 4...",4.127932,838.805970,81.950000,"[{""cluster_name"": ""Dr. Rajkumar Road, Rajaji N...","{""count"": 1, ""pois"": [{""id"": ""ChIJCUMusVM9rjsR...",1706512950,"{""companies"": 0.11, ""vibrancy"": 0.16, ""transpo...","{""company_index"": 3.89, ""connectivity_index"": ...","{""clinic"": 25, ""college"": 7, ""hospital"": 57, ""..."
2,374c8386-5837-43b8-8409-c456849263e7,130009_776325,Orion Avenue - Second Cross N. Thyagaraju Layo...,13.000957,77.632571,POLYGON ((77.62847982126846 13.000488913937247...,"Second Cross N. Thyagaraju Layout, Sathya Naga...",i500mtb,"[{""id"": ""ChIJAfIetiRpUjoRKDXrDBoNP-0"", ""name"":...","{""count"": 24, ""avg_number_of_reviews_per_day"":...",...,"{""age_0_19"": 4633.231310760981, ""age_20_34"": 4...",3.616629,500.000000,82.470000,[],"{""count"": 0, ""pois"": []}",1706684037,"{""companies"": 0.11, ""vibrancy"": 0.16, ""transpo...","{'company_index': 4.01, 'connectivity_index': ...","{""clinic"": 27, ""hospital"": 5, ""pharmacy"": 8, ""..."
3,2ef72172-e65c-47df-9f06-2a98c9f1fa94,130009_776325,Orion Avenue - Second Cross N. Thyagaraju Layo...,13.000957,77.632571,"POLYGON ((77.644571 13.032047, 77.643571 13.03...","Second Cross N. Thyagaraju Layout, Sathya Naga...",i15mind,"[{""id"": ""18767303"", ""name"": ""Imperio Restauran...","{""count"": 674, ""avg_number_of_reviews_per_day""...",...,"{""age_0_19"": 164433.15741358572, ""age_20_34"": ...",3.506014,498.559078,82.470000,"[{""cluster_name"": ""8th Main 3rd Phase Peenya I...","{""count"": 0, ""pois"": []}",1706684064,"{""companies"": 0.11, ""vibrancy"": 0.16, ""transpo...","{'company_index': 4.01, 'connectivity_index': ...","{""clinic"": 1313, ""college"": 135, ""hospital"": 3..."
4,5fca8372-db67-4e32-9c34-17818bca7c19,130552_777638,"Orion Uptown - Old Madras Road, Kattanallur,Sa...",13.055260,77.763894,"POLYGON ((77.773894 13.116797, 77.772949 13.11...","Old Madras Road, Kattanallur,Sannatammanahalli",i15mind,"[{""id"": ""ChIJdUCMjRcPrjsRjZ7tgReBR3M"", ""name"":...","{""count"": 85, ""avg_number_of_reviews_per_day"":...",...,"{""age_0_19"": 37307.81912734546, ""age_20_34"": 3...",3.134148,535.593220,40.173600,[],"{""count"": 2, ""pois"": [{""id"": ""ChIJfU4i1c0PrjsR...",1706691413,"{""companies"": 0.11, ""vibrancy"": 0.16, ""transpo...","{'company_index': 2.15, 'connectivity_index': ...","{""clinic"": 187, ""college"": 67, ""hospital"": 89,..."
5,ed374c4c-6dc3-473b-867e-ff707e347939,130552_777638,"Orion Uptown - Old Madras Road, Kattanallur,Sa...",13.055260,77.763894,"POLYGON ((77.75979674534983 13.05479248128713,...","Old Madras Road, Kattanallur,Sannatammanahalli",i500mtb,"[{""id"": ""ChIJu_szAwoPrjsR

In [75]:
def get_dictionary_value(d, keys):
    keys = keys.split(".")
    for i in keys:
        d = d.get(i)
    return d

In [76]:
project_info = U.get_project_info(id='130631_776205')
project_info = project_info.query("catchment_type!='i1000mtb'").to_dict('records')
project_info=[{k:json.loads(v) if "{" in str(v) else v for k,v in project.items()} for project in project_info]
project_info={project['catchment_type']:project for project in project_info}

In [77]:
data=dict(data=project_info)

In [85]:
data['data']['i500mtb']['avg_cost_for_two']

0.0

In [78]:
get_dictionary_value(data,'data.i500mtb.average_cost_for_two')

In [108]:
dict(pd.DataFrame(data['data']['i15mind']['competition']['pois']).groupby('type').id.count().reset_index().values.tolist())

{'anchor_competitor': 53, 'secondary_competitor': 141}

In [29]:
all_reports.drop_duplicates(subset=['id'])[['id','lat','lng']].to_dict('records')

[{'id': '130110_775547', 'lat': 13.011026921074786, 'lng': 77.55473637957641},
 {'id': '130009_776325', 'lat': 13.0009569, 'lng': 77.6325712},
 {'id': '130552_777638', 'lat': 13.0552603, 'lng': 77.7638939},
 {'id': '130631_776205', 'lat': 13.06317, 'lng': 77.620569},
 {'id': '128873_775969', 'lat': 12.887373, 'lng': 77.596901}]

In [31]:
from revenue_score import generate_revenue_score

In [34]:
query = '''UPDATE ind_site_reports_v2
SET
    revenue_score=:revenue_score,
    location_score_weights=:location_score_weights,
    grouped_indexes=:grouped_indexes
WHERE  id=:id;
'''
weights=json.dumps({
 'affluence_index': 0.19,
 'apartments_index': 0.19,
  'fashion_index': 0.19,
 'vibrancy': 0.14,
 'healthcare_index': 0.10,
 'company_index': 0.10,
 'malls_index': 0.05,
 'supermarket_index': 0.04,
 })
if __name__ == '__main__':
    con, cur = U.connect_to_db()

    for proj in all_reports.drop_duplicates(subset=['id'])[['id','lat','lng']].to_dict('records'):
        site_id=proj['id']
        lat=proj['lat']
        lng=proj['lng']
        travel_mode = 'driving'
        cost_type = 'time'
        cost = 15
        revenue_score, grouped_indexs = generate_revenue_score(lat, lng, travel_mode, cost_type, cost)
        grouped_indexs=json.dumps(grouped_indexs)
        query_params=dict(revenue_score=revenue_score, grouped_indexes=grouped_indexs,location_score_weights=weights,id=site_id)
        cur.execute(query, query_params)
        con.commit()
    cur.close()
    con.close()
